# Functions for Courses File

In [2]:
import pandas as pd
import sqlite3
import courses
from importlib import reload

In [35]:
# Run this if you change the 'courses.py' file but have already created the database in this notebook
reload(courses);

In [36]:
# connect to the database
# use create = True only the very first time running
db = courses.CoursesDB('courses.db', create = True) 

In [24]:
# run this box on your first time connecting to the database
# OR if you change something in the 'courses.py' file related to building the tables
db.drop_all_tables(are_you_sure = True)
db.build_tables()

In [25]:
# load the data for any meets you want to look at
# use db.load_results('link')
db.load_results('https://www.tfrrs.org/results/xc/23362/2024_Pirate_Cross_Country_Invitational_')
db.load_results('https://www.tfrrs.org/results/xc/25186/Panorama_Farms_Invitational')
db.load_results('https://tfrrs.org/results/xc/24854/2024_CAA_Cross_Country_Championship')
db.load_results('https://www.tfrrs.org/results/xc/23538/The_CNU_XC_Invitational')

In [26]:
# all the data together will look something like this
db.run_query('''SELECT * FROM tRunner
JOIN tRaceResult USING(runner_id)
JOIN tRace USING(race_id)''')

,runner_id,name,eligibility,school,race_id,raw_time,time,place,race,date
0,1,Peninah Mutisya,FR-1,Hampton,1,20:54.4,1254.4,1,2024 Pirate Cross Country Invitational,"October 4, 2024"
1,2,Kyra Holland,SR-4,William & Mary,1,21:07.1,1267.1,2,2024 Pirate Cross Country Invitational,"October 4, 2024"
2,3,Sofia Istnick,JR-3,William & Mary,1,21:35.8,1295.8,3,2024 Pirate Cross Country Invitational,"October 4, 2024"
3,4,Arianna DeBoer,SR-4,William & Mary,1,21:39.3,1299.3,4,2024 Pirate Cross Country Invitational,"October 4, 2024"
4,5,Sarah McCartney,SR-4,Charleston Southern,1,21:42.4,1302.4,5,2024 Pirate Cross Country Invitational,"October 4, 2024"
...,...,...,...,...,...,...,...,...,...,...
543,427,Brooklynn Greene,FR-1,Regent University,4,40:48.5,2448.5,82,The CNU XC Invitational,"September 13, 2024"
544,428,Lindsey Tanner,FR-1,Regent University,4,40:49.3,2449.3,83,The CNU XC Invitational,"September 13, 2024"
545,429,Samiya Dell,FR-1,Virginia State,4,41:31.3,2491.3,84,The CNU XC Invitational,"September 13, 2024"
546,430,Skye Mitchell,FR-1,Virginia State,4,42:53.1,2573.1,85,The CNU XC Invitational,"September 13, 2024"


In [27]:
# see which races you have loaded
db.see_loaded_races()

,race_id,race,date
0,1,2024 Pirate Cross Country Invitational,"October 4, 2024"
1,2,Panorama Farms Invitational,"October 19, 2024"
2,3,2024 CAA Cross Country Championship,"November 1, 2024"
3,4,The CNU XC Invitational,"September 13, 2024"


In [30]:
# look up a race by a fragment of the name
db.course_lookup('Panorama')

,race_id,race,date
0,2,Panorama Farms Invitational,"October 19, 2024"


In [38]:
# look up a runner by a fragment of their name
db.runner_lookup('Anna')

,runner_id,name,eligibility,school
0,4,Arianna DeBoer,SR-4,William & Mary
1,32,Anna Berquist,SR-4,Davidson
2,37,Savannah Stoutt,SO-2,Col. of Charleston
3,54,Hannah Brown,SR-4,Milligan
4,70,Anna Conrad,FR-1,Winthrop
5,88,Leanna Lewis,SO-2,Norfolk State
6,110,Gianna Del Pizzo,FR-1,UCF
7,148,Brianna Dooney,FR-1,Winthrop
8,164,Hannah Morgan,SO-2,Davidson
9,166,Anna Jones,SR-4,Milligan


In [32]:
# find all meets that two runners ran in together (using their runner_id's)
db.find_races_in_common(1,2)

,race_id,race,date
0,1,2024 Pirate Cross Country Invitational,"October 4, 2024"
1,3,2024 CAA Cross Country Championship,"November 1, 2024"


In [37]:
# This function compares two courses specified by their race_id's.
# It will output the difference in seconds in average race times (difference), the ratio of average race times (ratio), and
# the number of runners in common between the two courses (NumCompared).
# The first course is used as a comparison point. 'difference' is the number of seconds faster or slower that the second course
# averages compared to the first course; a negative value for 'difference' means the second course was faster.
# 'ratio' is the number that times from the second course would need to be multiplied by in order to standardize them to the first
# course; the average time from the second course multiplied by 'ratio' should yield the average time from the first course.
# This function only compares times in runners who competed in both meets. The number of runners in common is shown as NumCompared.
db.compare_two_courses(1,2)

,Difference,Ratio,NumCompared
0,31.133333,1.022844,60
